In [1]:
from faultsDetectionDL.utils.config_loader import load_config, load_datasets_from_cfg
from matplotlib import pyplot as plt
import os
from ipywidgets import interact
import ipywidgets as widgets
import torch
from scipy.ndimage import gaussian_filter
import numpy as np
from functools import partial

In [2]:
os.chdir("../../")

In [13]:
train_data_dir = "models/configs/data_config/sample1.yaml"
valid_data_dir = "models/configs/data_config/sample2.yaml"
train_data_cfg = load_config(train_data_dir)
valid_data_cfg = load_config(valid_data_dir)

from LxGeoPyLibs.vision.image_transformation import Trans_Identity, Trans_Rot90, Trans_Rot180, Trans_Rot270, Trans_Flipud, Trans_fliplr, Trans_gaussian_noise, Trans_gamma_adjust, Trans_equal_hist
augmentation_transforms = None#[Trans_Identity(), Trans_Rot90(), Trans_Rot180(), Trans_Rot270(), Trans_fliplr(), Trans_gaussian_noise(), Trans_gamma_adjust(), Trans_equal_hist()]
    
train_dataset= load_datasets_from_cfg(train_data_cfg, augmentation_transforms=augmentation_transforms)
valid_dataset= load_datasets_from_cfg(valid_data_cfg, augmentation_transforms=augmentation_transforms)
train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1, num_workers=0, shuffle=False, drop_last=True)

valid_dataloader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=1, num_workers=0, shuffle=False, drop_last=True)
    

In [15]:

cnt = 256*256

def perc(idx):
    vals = train_dataset[idx][1].sum([1,2]) / cnt
    return vals


total_vals = torch.zeros_like(perc(0))
for idx in range(len(train_dataset)):
    total_vals += perc(idx)

sum(total_vals) / total_vals

tensor([ 1.0478, 65.1193, 33.0549])

In [16]:
sum(total_vals) / total_vals / sum(sum(total_vals) / total_vals)

tensor([0.0106, 0.6563, 0.3331])

In [5]:
def func(idx):
    ax = plt.imshow(valid_dataset[idx][0][:3].permute(1,2,0)/255)
    plt.show()
    

In [62]:
a=list(valid_dataloader)

x,y = [ torch.cat([a[i][j] for i in range(len(a))], 0) for j in (0,1) ]

In [6]:
chosen_dataset = train_dataset
print(chosen_dataset.datasets_lengths)
def func(idx):
    fig, axs = plt.subplots(1, 2, figsize=(15, 15))
    plt.subplot(1, 2, 1)
    axs[0].imshow(chosen_dataset[idx][0][:3].permute(1,2,0)/255)
    plt.subplot(1, 2, 2)
    axs[1].imshow(chosen_dataset[idx][0][:3].permute(1,2,0)/255)
    axs[1].imshow(chosen_dataset[idx][1][:3].permute(1,2,0), alpha=0.1)
    plt.show()

[1171, 2106, 580]


In [7]:
%matplotlib inline
interact(func, idx=widgets.IntSlider(min=0, max=len(chosen_dataset), step=1, value=0))

interactive(children=(IntSlider(value=0, description='idx', max=3857), Output()), _dom_classes=('widget-intera…

<function __main__.func(idx)>

In [7]:

%matplotlib qt5
a=np.transpose(valid_dataset[0][1],(1,2,0))
plt.imshow(gaussian_filter(a, sigma=0.6))


In [49]:
a=np.transpose(valid_dataset[0][1],(1,2,0))
def blur_fn(sigma, truncate):
    g=gaussian_filter(a, sigma=sigma, truncate=truncate, mode="constant")
    ax = plt.imshow(g/g.sum(axis=2)[:,:,None])
    plt.show()

In [50]:
%matplotlib qt5
interact(blur_fn, sigma=widgets.FloatSlider(min=0, max=2, step=0.01, value=0), truncate=widgets.IntSlider(min=0, max=50, step=1, value=0))

interactive(children=(FloatSlider(value=0.0, description='sigma', max=2.0, step=0.01), IntSlider(value=0, desc…

<function __main__.blur_fn(sigma, truncate)>